Entrenamiento para usar pýthon 

In [1]:
def convertToBase13(num):
    if num == 0:
        return "0"

    # Digits for base 13
    base13_digits = "0123456789ABC"  
    digits = ""
    positive = abs(num)
    
    while positive > 0:
        # Append digits/letters
        digits += base13_digits[positive % 13]  
        positive = positive // 13

    # Reverse the string at the end
    reversed_digits = digits[::-1]  
    if num < 0:
        return "-" + reversed_digits
    else:
        return reversed_digits

In [5]:
num = int(input("Escriba un numero"))
convertToBase13(num)

'A'

In [2]:
def func(x):
    if x == 1:
        def rv():
            print("X is equal to 1")
    else:
        def rv():
            print("X is not 1")

    return rv

In [5]:
new_func = func(2)
new_func

<function __main__.func.<locals>.rv()>

In [6]:
import inspect
from queue import Queue

print(inspect.getsource(Queue))

class Queue:
    '''Create a queue object with a given maximum size.

    If maxsize is <= 0, the queue size is infinite.
    '''

    def __init__(self, maxsize=0):
        self.maxsize = maxsize
        self._init(maxsize)

        # mutex must be held whenever the queue is mutating.  All methods
        # that acquire mutex must release it before returning.  mutex
        # is shared between the three conditions, so acquiring and
        # releasing the conditions also acquires and releases mutex.
        self.mutex = threading.Lock()

        # Notify not_empty whenever an item is added to the queue; a
        # thread waiting to get is notified then.
        self.not_empty = threading.Condition(self.mutex)

        # Notify not_full whenever an item is removed from the queue;
        # a thread waiting to put is notified then.
        self.not_full = threading.Condition(self.mutex)

        # Notify all_tasks_done whenever the number of unfinished tasks
        # drops to zero; th

# Validador de Agendamiento de Reuniones (Calendar Booking System)

## 📝 Descripción:
Construirás un sistema de agendamiento de reuniones que:

    - Permita a usuarios crear reuniones con fecha, hora de inicio y duración.

    - Valide que no haya solapamientos entre reuniones existentes.

    - Permita consultar las reuniones existentes de un día.

    - Permita cancelar reuniones.

## 🎯 Requisitos funcionales:
    
    - Crear reunión (schedule_meeting)

    - Parámetros: start_time, duration, description

    - Valida que no se cruce con ninguna otra existente.

    - Listar reuniones (list_meetings(date))

    - Devuelve lista ordenada por hora.

    - Cancelar reunión (cancel_meeting(id))

## 💡 Consideraciones técnicas:

    - Usa datetime para manejar fechas y horas.

    - Las reuniones duran una cantidad de minutos (duration).

    - Valida que una reunión no se cruce en el tiempo con otra.

In [1]:
from datetime import datetime, timedelta
from typing import List
import uuid

class Meeting:
    def __init__(self, start_time: datetime, duration: int, description: str):
        self.id = str(uuid.uuid4())
        self.start_time = start_time
        self.end_time = start_time + timedelta(minutes=duration)
        self.duration = duration
        self.description = description

    def __str__(self):
        return f"{self.description} ({self.start_time.strftime('%H:%M')} - {self.end_time.strftime('%H:%M')})"

class MeetingScheduler:
    def __init__(self):
        self.meetings: List[Meeting] = []

    def schedule_meeting(self, start: str, duration: int, description: str):
        start_dt = datetime.strptime(start, "%Y-%m-%d %H:%M")
        new_meeting = Meeting(start_dt, duration, description)

        for meeting in self.meetings:
            if self._overlaps(meeting, new_meeting):
                raise ValueError("La nueva reunión se cruza con una existente.")

        self.meetings.append(new_meeting)
        print(f"Reunión programada: {new_meeting}")

    def _overlaps(self, m1: Meeting, m2: Meeting) -> bool:
        return m1.start_time < m2.end_time and m2.start_time < m1.end_time

    def list_meetings(self, date: str):
        date_dt = datetime.strptime(date, "%Y-%m-%d").date()
        filtered = [m for m in self.meetings if m.start_time.date() == date_dt]
        for m in sorted(filtered, key=lambda x: x.start_time):
            print(f"[{m.id[:6]}] {m}")

    def cancel_meeting(self, meeting_id: str):
        before = len(self.meetings)
        self.meetings = [m for m in self.meetings if m.id != meeting_id]
        after = len(self.meetings)
        if before == after:
            print("ID no encontrado.")
        else:
            print("Reunión cancelada.")

In [3]:
calendar = MeetingScheduler()

calendar.schedule_meeting("2025-07-17 09:00", 60, "Reunión con cliente")
calendar.schedule_meeting("2025-07-17 10:30", 30, "Planificación interna")

Reunión programada: Reunión con cliente (09:00 - 10:00)
Reunión programada: Planificación interna (10:30 - 11:00)


In [5]:
calendar.list_meetings("2025-07-17")
# → Reunión con cliente (9:00 - 10:00), Planificación interna (10:30 - 11:00)

[57eb5e] Reunión con cliente (09:00 - 10:00)
[90a3e3] Planificación interna (10:30 - 11:00)


In [6]:
calendar.schedule_meeting("2025-07-17 09:30", 30, "Solapada")  
# → Error: se cruza con otra

ValueError: La nueva reunión se cruza con una existente.

# Diseña un sistema de detección de tasa de peticiones (Rate Limiter)

## 💼 Descripción:
Vas a construir un Rate Limiter, es decir, un componente que impide que un usuario (o IP, o servicio) haga demasiadas peticiones en un corto periodo de tiempo.

    - Este patrón es usado en producción por:

    - APIs públicas (OpenAI, Twitter, GitHub)

    - Servicios que protegen de abuso (login, spam, etc.)

## 🎯 Requisitos:
Solo se deben permitir N peticiones por usuario por ventana de tiempo T (por ejemplo, 5 peticiones cada 10 segundos).

    - Rechaza peticiones que excedan ese límite.

    - Opcional: Implementa variantes como sliding window o leaky bucket si quieres ir más lejos.

In [1]:
from collections import deque
from typing import Dict

class RateLimiter:
    def __init__(self, max_requests: int, window_seconds: int):
        self.max_requests = max_requests
        self.window_seconds = window_seconds
        self.user_requests: Dict[str, deque] = {}

    def allow_request(self, user_id: str, current_time: int) -> bool:
        """
        Registra y decide si la petición puede hacerse en el timestamp actual.
        """
        if user_id not in self.user_requests:
            self.user_requests[user_id] = deque()

        requests = self.user_requests[user_id]

        # Elimina peticiones fuera de la ventana de tiempo
        while requests and current_time - requests[0] >= self.window_seconds:
            requests.popleft()

        if len(requests) < self.max_requests:
            requests.append(current_time)
            return True
        return False

In [4]:
limiter = RateLimiter(max_requests=3, window_seconds=10)

#print(limiter.allow_request("user1", 1))   # True
#print(limiter.allow_request("user1", 2))   # True
#print(limiter.allow_request("user1", 3))   # True
#print(limiter.allow_request("user1", 4))   # False (excede el límite)
print(limiter.allow_request("user1", 12))  # True (ventana reseteada)


True


## Sistema de Clasificación de Productos con Búsqueda y Filtros Eficientes

### 📝 Descripción:
Vas a construir un motor de búsqueda y clasificación de productos (como los usados en Alibaba, JD.com, Pinduoduo), que:

1. Permite registrar productos con:

    - id, nombre, precio, categoría, rating, stock

2. Permite realizar búsquedas filtradas por:

    - Categoría

    - Precio mínimo/máximo

    - Rating mínimo

    - Palabras clave en el nombre

3. Permite ordenar los resultados por:

    - Precio (asc o desc)

    - Rating

    - Relevancia (match en nombre)

In [1]:
from typing import List, Optional

class Product:
    def __init__(self, pid: int, name: str, price: float, category: str, rating: float, stock: int):
        self.pid = pid
        self.name = name
        self.price = price
        self.category = category
        self.rating = rating
        self.stock = stock

    def __repr__(self):
        return f"{self.name} - ${self.price} - ⭐{self.rating}"

class ProductSearchEngine:
    def __init__(self):
        self.products: List[Product] = []

    def add_product(self, pid: int, name: str, price: float, category: str, rating: float, stock: int):
        self.products.append(Product(pid, name, price, category, rating, stock))

    def search(self,
               category: Optional[str] = None,
               min_price: Optional[float] = None,
               max_price: Optional[float] = None,
               min_rating: Optional[float] = None,
               keyword: Optional[str] = None,
               sort_by: Optional[str] = None,
               ascending: bool = False) -> List[Product]:

        results = self.products

        if category:
            results = [p for p in results if p.category.lower() == category.lower()]
        if min_price is not None:
            results = [p for p in results if p.price >= min_price]
        if max_price is not None:
            results = [p for p in results if p.price <= max_price]
        if min_rating is not None:
            results = [p for p in results if p.rating >= min_rating]
        if keyword:
            results = [p for p in results if keyword.lower() in p.name.lower()]

        if sort_by == "price":
            results.sort(key=lambda p: p.price, reverse=not ascending)
        elif sort_by == "rating":
            results.sort(key=lambda p: p.rating, reverse=not ascending)
        elif sort_by == "name":
            results.sort(key=lambda p: p.name)

        return results

In [2]:
store = ProductSearchEngine()

store.add_product(1, "Smartphone Xiaomi Redmi", 799, "Electrónica", 4.5, 10)
store.add_product(2, "Xiaomi Band 7", 199, "Electrónica", 4.8, 5)
store.add_product(3, "Lenovo Laptop IdeaPad", 1200, "Computación", 4.3, 7)

result = store.search(category="Electrónica", min_price=100, max_price=800, sort_by="rating")

# → Devuelve productos 2 y 1 ordenados por rating descendente

In [3]:
result

[Xiaomi Band 7 - $199 - ⭐4.8, Smartphone Xiaomi Redmi - $799 - ⭐4.5]

## Sistema de Reservas de Salas con Conflicto Mínimo (Smart Room Allocator)

### 📝 Descripción:
Vas a construir un sistema que administre reservas de salas en una empresa. Pero con una diferencia clave:
Si no hay sala disponible en el horario solicitado, el sistema debe encontrar la sala con el menor conflicto posible (tiempo libre más cercano) o sugerir una alternativa.

Este ejercicio está inspirado en los sistemas de calendarización de Yandex o Tinkoff.

### 🎯 Requisitos funcionales:
1. Puedes registrar salas (add_room(nombre)).

2. Puedes hacer una reserva (book_room(nombre_sala, fecha, hora_inicio, duración_minutos)).

3. Si no hay disponibilidad exacta:

    - Sugerir la sala con el menor tiempo de conflicto.

    - O decir que no hay forma de agendar hoy.

4. Puedes listar reservas por sala y fecha.

In [4]:
from datetime import datetime, timedelta
from typing import List, Dict

class Reservation:
    def __init__(self, start: datetime, duration: int):
        self.start = start
        self.end = start + timedelta(minutes=duration)

    def overlaps(self, other: 'Reservation') -> bool:
        return self.start < other.end and other.start < self.end

    def __str__(self):
        return f"{self.start.strftime('%H:%M')} - {self.end.strftime('%H:%M')}"

class RoomScheduler:
    def __init__(self):
        self.rooms: Dict[str, List[Reservation]] = {}

    def add_room(self, room_name: str):
        if room_name in self.rooms:
            print("La sala ya existe.")
        else:
            self.rooms[room_name] = []

    def book_room(self, room_name: str, date: str, time: str, duration: int):
        if room_name not in self.rooms:
            print(f"Sala {room_name} no registrada.")
            return

        start = datetime.strptime(f"{date} {time}", "%Y-%m-%d %H:%M")
        new_reservation = Reservation(start, duration)

        for res in self.rooms[room_name]:
            if res.overlaps(new_reservation):
                print(f"⚠️ Conflicto en {room_name} a las {res}")
                self.suggest_alternative(room_name, date, duration)
                return

        self.rooms[room_name].append(new_reservation)
        print(f"✅ Reserva confirmada en {room_name} de {new_reservation}")

    def suggest_alternative(self, room_name: str, date: str, duration: int):
        print("🔍 Buscando alternativa...")
        for alt_room, reservations in self.rooms.items():
            if alt_room == room_name:
                continue
            start_time = datetime.strptime(f"{date} 08:00", "%Y-%m-%d %H:%M")
            end_time = datetime.strptime(f"{date} 18:00", "%Y-%m-%d %H:%M")

            while start_time + timedelta(minutes=duration) <= end_time:
                trial = Reservation(start_time, duration)
                if all(not r.overlaps(trial) for r in reservations):
                    print(f"💡 Alternativa: reservar {alt_room} a las {start_time.strftime('%H:%M')}")
                    return
                start_time += timedelta(minutes=15)
        print("❌ No hay alternativa disponible hoy.")

    def list_reservations(self, room_name: str, date: str):
        print(f"📋 Reservas para {room_name} en {date}:")
        for r in sorted(self.rooms.get(room_name, []), key=lambda x: x.start):
            if r.start.strftime("%Y-%m-%d") == date:
                print(f" - {r}")

In [5]:
scheduler = RoomScheduler()
scheduler.add_room("Sala A")
scheduler.add_room("Sala B")

scheduler.book_room("Sala A", "2025-07-17", "09:00", 60)
scheduler.book_room("Sala A", "2025-07-17", "10:00", 30)

# Intento hacer reserva que se cruza
scheduler.book_room("Sala A", "2025-07-17", "09:30", 30)
# → Sugerencia: usar Sala B o mover a 10:30

scheduler.list_reservations("Sala A", "2025-07-17")

✅ Reserva confirmada en Sala A de 09:00 - 10:00
✅ Reserva confirmada en Sala A de 10:00 - 10:30
⚠️ Conflicto en Sala A a las 09:00 - 10:00
🔍 Buscando alternativa...
💡 Alternativa: reservar Sala B a las 08:00
📋 Reservas para Sala A en 2025-07-17:
 - 09:00 - 10:00
 - 10:00 - 10:30
